<a href="https://colab.research.google.com/github/aayesha-coder/Bert-Fine-tuning-for-answering-the-questions/blob/main/Bert_fine_tune_for_sentece_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=c36a9ab89f57d9625182ddabc91e4bf9c6f2d62c107774f96aab7ca7a0284e0c
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
import wget
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'
wget.download(url, './cola_public_1.1.zip')


'./cola_public_1.1.zip'

In [ ]:
! unzip cola_public_1.1.zip

Archive:  cola_public_1.1.zip
   creating: cola_public/
  inflating: cola_public/README      
   creating: cola_public/tokenized/
  inflating: cola_public/tokenized/in_domain_dev.tsv  
  inflating: cola_public/tokenized/in_domain_train.tsv  
  inflating: cola_public/tokenized/out_of_domain_dev.tsv  
   creating: cola_public/raw/
  inflating: cola_public/raw/in_domain_dev.tsv  
  inflating: cola_public/raw/in_domain_train.tsv  
  inflating: cola_public/raw/out_of_domain_dev.tsv  


In [ ]:
import pandas as pd

df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
print(df.shape)
df.sample(5)

(8551, 4)


,sentence_source,label,label_notes,sentence
4560,ks08,0,*,It has rain every day for the last week.
5050,ks08,1,NaN,It surprised me that you came early.
8268,ad03,0,*,Jason asked whether that the potion was ready
4209,ks08,1,NaN,Most of the milk your baby consumes during bre...
1253,r-67,1,NaN,You're going to hurt yourself one of these days.


In [ ]:
df.sample(5)[['sentence', 'label']]

,sentence,label
6258,Robert knows it is essential that he is well b...,1
5110,It's the writer that gets you so involved.,1
7037,Which article did Terry file without reading?,1
7455,I wonder has whether Mary worked for Microsoft.,0
7591,Mary considers John proud of herself.,0


In [ ]:
sentence_list = df.sentence.values
label_list = df.label.values

print(sentence_list)
print(label_list)

["Our friends won't buy this analysis, let alone the next one we propose."
 "One more pseudo generalization and I'm giving up."
 "One more pseudo generalization or I'm giving up." ...
 'It is easy to slay the Gorgon.'
 'I had the strangest feeling that I knew you.'
 'What all did you get for Christmas?']
[1 1 1 ... 1 1 1]


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 63.0 MB/s 
     |████████████████████████████████| 6.6 MB 48.7 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer

b_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
print('Sample sentenece: ', sentence_list[0])


Sample sentenece:  Our friends won't buy this analysis, let alone the next one we propose.


In [ ]:
tokens = b_tokenizer.tokenize(sentence_list[0])
print('Sentence tokens: ', tokens)


Sentence tokens:  ['our', 'friends', 'won', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.']


In [ ]:
token_codes = b_tokenizer.convert_tokens_to_ids(b_tokenizer.tokenize(sentence_list[0]))
print('Token codes: ', token_codes)

Token codes:  [2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012]


In [ ]:
max_sent_len = 0

for sentence in sentence_list:
    # three steps......
    #..................1)tokens formation.....2)token to codes conversion.....3)add special tokens.....
    token_codes = b_tokenizer.encode(sentence, add_special_tokens=True)

    # get maximum length of sentence in dataset
    max_sent_len = max(max_sent_len, len(token_codes))

print('The longest sentence size: ', max_sent_len)

The longest sentence size:  47


In [ ]:
token_codes = []
attention_masks = []

# For each sentence
for sentence in sentence_list:
    token_code_dict = b_tokenizer.encode_plus(         # convert data to Model format
                        sentence,                      # encode sentence
                        add_special_tokens = True,     # add special tokens'[CLS]' and '[SEP]'
                        max_length = max_sent_len,     # specify max length of sentence
                        pad_to_max_length = True,      # add padding tokens till max length
                        return_attention_mask = True,  # assign attnention masks
                        return_tensors = 'pt',         # return pytorch tensors
                   )
    
    # Make a list of encoded sentences  
    token_codes.append(token_code_dict['input_ids']) 
    # Make a list of sentence attention masks
    attention_masks.append(token_code_dict['attention_mask'])

print(token_codes[0])
print(attention_masks[0])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


tensor([[  101,  2256,  2814,  2180,  1005,  1056,  4965,  2023,  4106,  1010,
          2292,  2894,  1996,  2279,  2028,  2057, 16599,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch

token_codes_tens = torch.cat(token_codes, dim=0)               # Make a tensor of encoded sentences to concatenate all encoded sentences 
attention_masks_tens = torch.cat(attention_masks, dim=0)   # Make a tensor of attention masks by concatenation
labels_tens = torch.tensor(label_list)                         # Make a tensor of labels to concatenate all labels


print('Sentence: ', sentence_list[0])
print('Encoded sentence: ', token_codes_tens[0])
print('Sentence attention masks: ', attention_masks_tens[0])
print('Sentence label: ', labels_tens[0] )

Sentence:  Our friends won't buy this analysis, let alone the next one we propose.
Encoded sentence:  tensor([  101,  2256,  2814,  2180,  1005,  1056,  4965,  2023,  4106,  1010,
         2292,  2894,  1996,  2279,  2028,  2057, 16599,  1012,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])
Sentence attention masks:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Sentence label:  tensor(1)


In [ ]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(token_codes_tens, attention_masks_tens, labels_tens)

In [ ]:
dataset_size = len(dataset)
train_data_size = int(0.9 * len(dataset))
val_data_size = dataset_size - train_data_size

print('Total data size: ', dataset_size)
print('Training data size: ', train_data_size)
print('Validation data size: ', val_data_size)

Total data size:  8551
Training data size:  7695
Validation data size:  856


In [ ]:
#............Randomly generate data samples..............
train_data, val_data = random_split(dataset, [train_data_size, val_data_size])

print(train_data[0])
print(val_data[0])


(tensor([  101,  1996,  2911,  1005,  1055, 10186,  2000,  8145,  1996,  5427,
         2001,  2200, 14386,  3560,  1012,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor(0))
(tensor([  101,  2122,  2024,  1996,  2477,  2005,  2029,  2000,  2022, 18836,
         1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_dataloader = DataLoader(                         # Get dataloader for training dataset
            train_data,                                # The training dataset
            sampler = RandomSampler(train_data), # Random batches of training data
            batch_size = batch_size                    # Each batch size for training
        )

val_dataloader = DataLoader(                           # Get dataloader for validation dataset
            val_data,                                  # The validation dataset
            sampler = SequentialSampler(val_data), # Sequential batches for validation
            batch_size = batch_size                    # Each batch size for validation
        )

In [ ]:
from transformers import BertForSequenceClassification

pretrain_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",           # pre-trained BERT with uncased vocabulary
    num_labels = 2,                # for two-class classification 
    output_attentions = False,     # does not output attention weights
    output_hidden_states = False,  # does not output hidden-states.
)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from transformers import AdamW, BertConfig
optimizer = AdamW(pretrain_model.parameters(),   # optimizer AdamW
                  lr = 2e-5,                     # learning rate
                  eps = 1e-8                     # Adam epsilon value
                )


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
num_epochs = 4

#........no. of total training steps.......
total_train_steps = len(train_dataloader) * num_epochs



In [ ]:
from transformers import get_linear_schedule_with_warmup
#.......learning rate scheduler...........
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_train_steps)

In [ ]:
import random
import numpy as np

seed_value = 42

random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)


In [ ]:
training_output = []
    
#----------------------Training-----------------------    
# For each epoch--------------------------------------
for each_epoch in range(0, num_epochs):
    print('........... Epoch {0} / {1} ...........'.format(each_epoch + 1, num_epochs))
    print('Training is going on...')
    training_loss = 0

    # put model in training mode----------------------
    pretrain_model.train()                                                    

    # process each batch in an epoch------------------
    for step, batch in enumerate(train_dataloader):                           
        batch_token_codes = batch[0]
        batch_attention_masks = batch[1]
        batch_labels = batch[2]
        pretrain_model.zero_grad()                                #-------------set gradient to zero----------------
        resulting_output = pretrain_model(batch_token_codes,      #-------------get output from pre-train model-----
                           token_type_ids=None, 
                           attention_mask=batch_attention_masks, 
                           labels=batch_labels,
                           return_dict=True)

        batch_loss = resulting_output.loss                        #-------------get loss in forward pass-----------
        batch_logits = resulting_output.logits                    #-------------get prediction scores--------------
        training_loss += batch_loss.item()                        #-------------get total epoch loss---------------
        batch_loss.backward()                                     #-------------get loss in backward pass----------
        torch.nn.utils.clip_grad_norm_(pretrain_model.parameters(), 1.0) #------normalize gradient score-----------
        optimizer.step()                                          #-------------take optimizer-specified step------                                       
        scheduler.step()                                          #-------------take step using lr scheduler-------
    average_train_loss = training_loss / len(train_dataloader)    #-------------calculate average training loss----      

    print('  Average training loss: {0:.3f}'.format(average_train_loss))
    training_output.append(                                       #------------list all training results-----------
        {
            'Epoch': each_epoch + 1,
            'Training Loss': average_train_loss
        }
    )


........... Epoch 1 / 4 ...........
Training is going on...
  Average training loss: 0.497
........... Epoch 2 / 4 ...........
Training is going on...
  Average training loss: 0.298
........... Epoch 3 / 4 ...........
Training is going on...
  Average training loss: 0.179
........... Epoch 4 / 4 ...........
Training is going on...
  Average training loss: 0.129


In [ ]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:

val_output = []

#------------------------Validation------------------------
# For each epoch-------------------------------------------
for each_epoch in range(0, num_epochs):     
    print('........... Epoch {0} / {1} ...........'.format(each_epoch + 1, num_epochs))
    print('Validation is going on...')
    validation_loss = 0
    validation_accuracy = 0   
    nb_eval_steps = 0
    
    # put model in evaluation mode-------------------------
    pretrain_model.eval()

    # process each batch in an epoch-----------------------
    for batch in val_dataloader:         
        batch_token_codes = batch[0]
        batch_attention_masks = batch[1]
        batch_labels = batch[2]
        with torch.no_grad():                                            #---------------set gradient to zero----------------
            resulting_output = pretrain_model(batch_token_codes,         #---------------get output from pretrain model------
                               token_type_ids=None,   
                               attention_mask=batch_attention_masks,
                               labels=batch_labels,
                               return_dict=True)
        batch_loss = resulting_output.loss                               #---------------get loss in forward pass------------
        batch_logits = resulting_output.logits                           #---------------get prediction scores---------------
        validation_loss += batch_loss.item()                             #---------------get epoch evaluation loss-----------
        validation_accuracy += flat_accuracy(batch_logits, batch_labels)      #---------------get epoch evaluation accuracy-------
    average_val_accuracy = validation_accuracy / len(val_dataloader)     #---------------get average epoch accuracy----------
    average_val_loss = validation_loss / len(val_dataloader)             #---------------get average epoch loss--------------

    print('  Average Validation Loss : {0:.3f}'.format(average_val_loss))
    print('  Average Validation Accuracy : {0:.3f}'.format(average_val_accuracy))
    val_output.append(                                                   #---------------list all evaluation results---------
        {
            'Epoch': each_epoch + 1,
            'Validation Loss': average_val_loss,
            'Validation Accuracy.': average_val_accuracy,
        }
    )

In [ ]:
import os

model_saving_dir = './finetune_model/'                                                           #------------location to save model-------------------
model_to_save = pretrain_model.module if hasattr(pretrain_model, 'module') else pretrain_model   #------------for parallel/non-parallel training-------
model_to_save.save_pretrained(model_saving_dir)                                                  #------------saving of model--------------------------
b_tokenizer.save_pretrained(model_saving_dir)
